In [ ]:
!pip install tensorflow_addons
# !pip install tensorflow==2.8
# !apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

import tensorflow as tf
print(tf.__version__)
print(tf.test.is_built_with_cuda())  # cuda로 빌드되는지 확인
print(tf.test.is_built_with_gpu_support())  # cuda와 같은 gpu로 빌드되는지 확인
print(tf.test.gpu_device_name())  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2.8.2
True
True
/device:GPU:0


In [ ]:
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
import os
import random

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow import keras
from google.colab import drive
from glob import glob

from keras import backend as K
from keras.utils import io_utils
from tensorflow.python.platform import tf_logging as logging

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/collage/segmentation')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
rs = 1234
IMAGE_SIZE = 512
BATCH_SIZE = 8
NUM_CLASSES = 1

def seed_everything(seed):
    random.seed(seed)
    # os.environ['PYTHONHASHSEED'] = str(seed)
    # os.environ['TF_DETERMINISTIC_OPS'] = '1'
    np.random.seed(seed)
    tf.random.set_seed(seed)
seed_everything(rs)

In [ ]:
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/collage/segmentation/data/rawdata/"

all_images = sorted(glob(os.path.join(DATA_DIR, "humanparsing/val/Images/*"))+glob(os.path.join(DATA_DIR, "humanparsing/train/Images/*"))+glob(os.path.join(DATA_DIR, "fashion/JPEGImages/*")))
all_masks = sorted(glob(os.path.join(DATA_DIR, "humanparsing/val/Human/*"))+glob(os.path.join(DATA_DIR, "humanparsing/train/Human/*"))+glob(os.path.join(DATA_DIR, "fashion/SegmentationClassAug/*")))

val_size = 5000
train_images, val_images, train_masks, val_masks  = train_test_split(all_images, all_masks, test_size=val_size, random_state=rs)

In [ ]:
def read_image(image_path, mask=False):
  image = tf.io.read_file(image_path)
  image = tf.image.decode_png(image, channels=3)
  image.set_shape([None, None, 3])
  image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
  if not mask:
      image = image / 255
  return image

def load_data(image_list, mask_list):
  image = read_image(image_list)
  mask = read_image(mask_list, mask=True)
  return image, mask

def random_crop(input_image, real_image, size):
  stacked_image = tf.stack([input_image, real_image], axis=0)
  cropped_image = tf.image.random_crop(
      stacked_image, size=[2, size, size, 3])
  img,msk = cropped_image[0], cropped_image[1]
  return img,msk

def augment(input_image, input_mask):

  if tf.random.uniform(()) > 0.3:
      ratio = tf.random.uniform(shape=[],minval=400, maxval=440, dtype=tf.int32)
      input_image, input_mask = random_crop(input_image, input_mask, ratio)
      input_image = tf.image.resize(input_image, (IMAGE_SIZE, IMAGE_SIZE))
      input_mask = tf.image.resize(input_mask, (IMAGE_SIZE, IMAGE_SIZE))

  input_image = tf.image.random_brightness(input_image, 0.2)

  saturation_factor = tf.random.uniform((),0,2)
  input_image = tf.image.adjust_saturation(input_image, saturation_factor)

  if tf.random.uniform(()) > 0.5:
      input_image = tf.image.flip_left_right(input_image)
      input_mask = tf.image.flip_left_right(input_mask)

  rot_factor = tf.cast(tf.random.uniform(shape=[],minval=-6, maxval=6, dtype=tf.int32), tf.float32)
  angle = np.pi/(12*6) *rot_factor
  input_image = tfa.image.rotate(input_image, angle)
  input_mask = tfa.image.rotate(input_mask, angle)

  return input_image, input_mask

def cast_mask(input_image, input_mask):
  input_mask = tf.math.reduce_sum(input_mask, axis=2)
  input_mask = input_mask[..., tf.newaxis]
  input_mask = tf.cast(input_mask>0, dtype=tf.float32)
  return input_image,input_mask

def data_generator(image_list, mask_list, augument=True):
  dataset = tf.data.Dataset.from_tensor_slices((image_list, mask_list))
  dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
  if augument:
    dataset = dataset.map(augment)
  dataset = dataset.map(cast_mask)
  dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
  return dataset

train_dataset = data_generator(train_images, train_masks)
print("Train Dataset:", train_dataset)

val_dataset = data_generator(val_images, val_masks, augument=False)
print("Validation Dataset:", val_dataset)

Train Dataset: <BatchDataset element_spec=(TensorSpec(shape=(8, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(8, 512, 512, 1), dtype=tf.float32, name=None))>
Validation Dataset: <BatchDataset element_spec=(TensorSpec(shape=(8, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(8, 512, 512, 1), dtype=tf.float32, name=None))>


In [ ]:
def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output

def SE_block(input_tensor, reduction_ratio=8):
    ch_input = K.int_shape(input_tensor)[-1]
    ch_reduced = ch_input//reduction_ratio  
    # Squeeze
    x = layers.GlobalAveragePooling2D()(input_tensor)
    # Excitation
    x = layers.Dense(ch_reduced, kernel_initializer='he_normal', activation='relu', use_bias=False)(x) 
    x = layers.Dense(ch_input, kernel_initializer='he_normal', activation='sigmoid', use_bias=False)(x)
    x = layers.Reshape((1, 1, ch_input))(x)
    x = layers.Multiply()([input_tensor, x])
    
    return x

In [ ]:
def DeeplabV3Plus(image_size, num_classes, find_tunebackbone=False):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    effiucientnetV2M = keras.applications.EfficientNetV2M(
        weights="imagenet", include_top=False, input_tensor=model_input
    )
    if not find_tunebackbone:
      for layer in effiucientnetV2M.layers:
        if '_bn' not in layer.name:
          layer.trainable = False
    x = effiucientnetV2M.get_layer("block6a_expand_activation").output
    x = DilatedSpatialPyramidPooling(x)
    # x = SE_block(x)
    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = effiucientnetV2M.get_layer("block2e_add").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same", activation='sigmoid')(x)
    return keras.Model(inputs=model_input, outputs=model_output)

In [ ]:
class unfreeze_layers_reduce_lr(tf.keras.callbacks.ReduceLROnPlateau):

  def __init__(self,
               monitor='val_loss',
               factor=0.1,
               patience=10,
               verbose=0,
               mode='auto',
               min_delta=1e-4,
               cooldown=0,
               min_lr=0,
               freezed=False,
               **kwargs):
      super().__init__(monitor, factor, patience, verbose, mode, min_delta, cooldown, min_lr)
      self.freezed = False


  def on_epoch_end(self, epoch, logs=None):
    logs = logs or {}
    logs['lr'] = K.get_value(self.model.optimizer.lr)
    current = logs.get(self.monitor)
    if current is None:
      logging.warning('Learning rate reduction is conditioned on metric `%s` '
                      'which is not available. Available metrics are: %s',
                      self.monitor, ','.join(list(logs.keys())))
    else:
      if self.in_cooldown():
        self.cooldown_counter -= 1
        self.wait = 0

      if self.monitor_op(current, self.best):
        self.best = current
        self.wait = 0

      elif not self.in_cooldown():
        self.wait += 1
        if self.wait >= self.patience:
          
          if not self.freezed:
            for layer in self.model.layers:
                      layer.trainable = True
            self.freezed=True
            old_lr = K.get_value(self.model.optimizer.lr)
            new_lr = old_lr * 0.1
            K.set_value(self.model.optimizer.lr, new_lr)
            io_utils.print_msg(
                    f'\nEpoch {epoch +1}: '
                    f'Unfreezing backbone layers and reducing learning rate to {new_lr}')
            self.cooldown_counter = self.cooldown
            self.wait = 0

          else:
            old_lr = K.get_value(self.model.optimizer.lr)
            if old_lr > np.float32(self.min_lr):
              new_lr = old_lr * self.factor
              new_lr = max(new_lr, self.min_lr)
              K.set_value(self.model.optimizer.lr, new_lr)
              if self.verbose > 0:
                io_utils.print_msg(
                    f'\nEpoch {epoch +1}: '
                    f'ReduceLROnPlateau reducing learning rate to {new_lr}.')
              self.cooldown_counter = self.cooldown
              self.wait = 0

class CustomSaver(keras.callbacks.Callback):
  def __init__(self, save_path, save_name, frequency):
    self.save_path = save_path
    self.save_name = save_name
    self.frequency = frequency

  def on_epoch_end(self, epoch, logs={}):
      if (epoch+1) % self.frequency == 0 :
          name = self.save_path + self.save_name + f'{epoch+1:03d}.h5'
          self.model.save(name)
          io_utils.print_msg(
                    f'\nEpoch {epoch + 1}:'
                    f'saving model to {name}')

model_path, model_name = './model_weight/DLv3+sgd0.01+freezed/', 'sgd0.01_epoch_'
model_path_best = './model_weight/DLv3+sgd0.01+freezed/sgd0.01.h5'
csv_name = './model_weight/DLv3+sgd0.01+freezed/sgd0.01.csv'

custom_saver = CustomSaver(model_path, model_name ,frequency=5)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1, mode='min', min_delta=1e-4, cooldown=0, min_lr=0.0001,)
save_best = tf.keras.callbacks.ModelCheckpoint(model_path_best, monitor='val_loss', verbose=0, save_best_only=True,)
write_log = tf.keras.callbacks.CSVLogger(csv_name, separator=',', append=True)

cb_list = [custom_saver, reduce_lr, save_best, write_log]

In [ ]:
model = DeeplabV3Plus(image_size=IMAGE_SIZE, num_classes=NUM_CLASSES)
# model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/collage/segmentation/model_weight/DLv3+sgd0.01+unfreezesgd0.01_epoch_020.h5')
loss = keras.losses.BinaryCrossentropy()
model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True),
    loss=loss,
    metrics=["accuracy", tf.keras.metrics.BinaryIoU(),],
)

214212608/214201816 [==============================] - 1s 0us/step


In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset,
                    callbacks=cb_list, epochs=100)